In [5]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go

def fetch_data(ticker, start_date='2021-01-01', end_date='2024-01-01'):
    """Fetch historical data for the given ticker."""
    return yf.download(ticker, start=start_date, end=end_date)

def ewmac(price, vol, Lfast, Lslow):
    """
    Calculate the ewmac trading rule forecast, given a price, volatility and EWMA speeds Lfast and Lslow

    Assumes that 'price' and vol is daily data

    This version uses a precalculated price volatility, and does not do capping or scaling

    :param price: The price or other series to use (assumed Tx1)
    :type price: pd.Series

    :param vol: The daily price unit volatility (NOT % vol)
    :type vol: pd.Series aligned to price

    :param Lfast: Lookback for fast in days
    :type Lfast: int

    :param Lslow: Lookback for slow in days
    :type Lslow: int

    :returns: pd.Series -- unscaled, uncapped forecast
    """
    fast_ewma = price.ewm(span=Lfast, min_periods=1).mean()
    slow_ewma = price.ewm(span=Lslow, min_periods=1).mean()
    raw_ewmac = fast_ewma - slow_ewma
    return raw_ewmac / vol.ffill()


def calculate_ewmac_signals(data, parameters):
    """
    Calculate EWMAC signals for a given price series, volatility, and a list of parameters.

    :param price: The price series
    :type price: pd.Series

    :param volatility: The volatility series
    :type volatility: pd.Series

    :param parameters: List of tuples containing fast and slow parameters
    :type parameters: list of tuples

    :returns: pd.DataFrame -- DataFrame containing EWMAC signals for each parameter pair
    """
    for filter_speed in parameters:
        data[f'EWMAC_{filter_speed}'] = scale_cap_forecast(ewmac(data['Close'], data['Volatility'], filter_speed, 4*filter_speed))
    return data


def scale_cap_forecast(signals, cap=20):
    """
    Scale the forecast so it has an expected absolute average value of 10.

    :param signals: EWMAC signals.
    :return: Scaled signals.
    """
    signals = signals * 10 / signals.abs().mean()
    return signals.clip(-cap, cap)
    
def plot_position_size(position_size, price):
    """
    Plot position size against underlying price with vertical lines indicating periods of long and short positions.

    :param position_size: Position size data.
    :param price: Underlying price data.
    """
    fig = go.Figure()

    # Plot underlying price
    fig.add_trace(go.Scatter(x=price.index, y=price, mode='lines', name='Price', line=dict(color='blue')))

    # Create a separate y-axis for position size
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size, mode='lines', name='Position Size', line=dict(color='red'), yaxis='y2'))

    # Add fill areas for long and short positions
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size * (position_size > 0), mode='lines', fill='tozeroy', name='Long Position', fillcolor='rgba(0,255,0,0.3)', line=dict(width=0), hoverinfo='skip', yaxis='y2'))
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size * (position_size < 0), mode='lines', fill='tozeroy', name='Short Position', fillcolor='rgba(255,0,0,0.3)', line=dict(width=0), hoverinfo='skip', yaxis='y2'))

    # Add vertical lines for direction change
    for i in range(1, len(position_size)):
        if position_size.iloc[i] != 0 and position_size.iloc[i - 1] == 0:
            fig.add_shape(type="line", xref="x", yref="paper", x0=position_size.index[i], x1=position_size.index[i], y0=0, y1=1, line=dict(color="green" if position_size[i] > 0 else "red", width=1), opacity=0.1)

    # Set layout
    fig.update_layout(title='Position Size vs Underlying Price with Direction Change Indicators', xaxis_title='Date')

    # Ensure both y-axes share y=0
    fig.update_layout(yaxis=dict(title='Price', color='blue'),
                      yaxis2=dict(title='Position Size', color='red', overlaying='y', side='right'))
    
    fig.show()

def plot_position_size_vs_pnl(position_size, price):
    """
    Plot position size against PNL with vertical lines indicating periods of long and short positions.

    :param position_size: Position size data.
    :param pnl: Profit and Loss data.
    """
    pnl = (calculate_pnl(position_size, price)).cumsum()
    fig = go.Figure()

    # Plot PNL
    fig.add_trace(go.Scatter(x=pnl.index, y=pnl, mode='lines', name='PNL', line=dict(color='blue')))

    # Create a separate y-axis for position size
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size, mode='lines', name='Position Size', line=dict(color='red'), yaxis='y2'))

    # Add fill areas for long and short positions
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size * (position_size > 0), mode='lines', fill='tozeroy', name='Long Position', fillcolor='rgba(0,255,0,0.3)', line=dict(width=0), hoverinfo='skip', yaxis='y2'))
    fig.add_trace(go.Scatter(x=position_size.index, y=position_size * (position_size < 0), mode='lines', fill='tozeroy', name='Short Position', fillcolor='rgba(255,0,0,0.3)', line=dict(width=0), hoverinfo='skip', yaxis='y2'))

    # Add vertical lines for direction change
    for i in range(1, len(position_size)):
        if position_size.iloc[i] != 0 and position_size.iloc[i - 1] == 0:
            fig.add_shape(type="line", xref="x", yref="paper", x0=position_size.index[i], x1=position_size.index[i], y0=0, y1=1, line=dict(color="green" if position_size[i] > 0 else "red", width=1), opacity=0.1)

    # Set layout
    fig.update_layout(title='Position Size vs PNL with Direction Change Indicators', xaxis_title='Date')

    # Ensure both y-axes share y=0
    fig.update_layout(yaxis=dict(title='PNL', color='blue'),
                      yaxis2=dict(title='Position Size', color='red', overlaying='y', side='right'))
    
    fig.show()


In [266]:
def calculate_position_size(capped_signals, risk_target, volatility, price, capital, instrument_weights, fx_rate, idm):
    """
    Calculate the position size based on capped signals, risk target, and other parameters.

    :param capped_signals: DataFrame containing capped EWMAC signals.
    :param risk_target: Annualized risk target.
    :param volatility: Current volatility of the instrument.
    :param price: Current price of the instrument.
    :param capital: Total capital available for trading.
    :param instrument_weights: Weights of instruments in the portfolio.
    :param fx_rate: Exchange rate if applicable.
    :param IDM: Instrument Diversification Multiplier.
    :return: Position size.
    """
    multiplier = 1000
    # Calculate the average of capped signals
    average_signal = capped_signals.mean(axis=1)
    
    # Calculate risk-adjusted unrounded position
    position = average_signal * capital * idm * instrument_weights * risk_target / (10 * multiplier * price * fx_rate * volatility)
    
    return position

def calculate_pnl(position_size, prices, multiplier=1000):
    return (position_size * prices * 1000).replace(0.0, np.nan).dropna()

def calculate_backtest_metrics(data, capital, strat):
    """
    Calculate various backtest metrics based on position size and price data.

    :param position_size: Series containing position size data.
    :param price_data: Series containing price data.

    :return: Dictionary containing backtest metrics.
    """
    # Calculate P&L (Profit and Loss)
    pnl = calculate_pnl(data['Position Size'], data['Daily_Returns'])
    
    # Calculate cumulative P&L
    cumulative_pnl = pnl.cumsum()

    # Calculate Sharpe ratio ################
    sharpe_ratio = pnl.mean() / pnl.std() * (252 ** 0.5)

    # Calculate annualized return
    annualized_return = pnl.mean() * 252

    annualized_return_pct = annualized_return*100/capital

    # Additional metrics
    drawdowns = cumulative_pnl - cumulative_pnl.cummax()
    avg_drawdown = drawdowns.mean()

    # Calculate average position size
    avg_position = data['Position Size'].abs().mean()
    turnover = (data['Position Size'].diff().abs() / avg_position).sum()
    skew = pnl.skew()
    lower_tail = pnl.quantile(0.05)
    upper_tail = pnl.quantile(0.95)
    alpha = (cumulative_pnl.iloc[-1] / cumulative_pnl.std()) * (252 ** 0.5)
    
    return pd.DataFrame({
        "Initial Capital": capital,
        "Mean Annualised Return  %": annualized_return_pct,
        "Avg Drawdown %": avg_drawdown,
        "Annual Standard Deviation %": pnl.std() * (252 ** 0.5),
        "Sharpe Ratio": sharpe_ratio,
        "Turnover": turnover,
        "Skew": skew,
        "Lower Tail (5th percentile)": lower_tail,
        "Upper Tail (95th percentile)": upper_tail,
        # "Alpha": alpha,
        "Cumulative PnL": cumulative_pnl.iloc[-1]
    }, index=[strat]).T

def backtest(ticker = 'BZ=F', start_date = '2020-01-01', end_date = '2024-01-01', parameters = [4, 8, 16, 32, 64],
             risk_target = 0.1, capital = 100000, plot=True):
    strat = "EWMAC_"  + '_'.join([str(i) for i in parameters])
    instrument_weights = 1 # Example instrument weights
    fx_rate = 1.0  # Example exchange rate if applicable
    idm = 1  # Example Instrument Diversification Multiplier

    # Fetch data
    data = fetch_data(ticker, start_date, end_date)

    # Calculate daily returns
    data['Daily_Returns'] = data['Close'].pct_change()

    # Calculate volatility
    data['Volatility'] = data['Daily_Returns'].ewm(span=30, min_periods=0).std()

    # Calculate EWMAC signals (scaled and capped)
    data = calculate_ewmac_signals(data, parameters)
    signal_cols = [i for i in data.columns if "EWM" in i]

    # Calculate position size
    data["Position Size"] = calculate_position_size(data[signal_cols], risk_target, data['Volatility'], data['Close'], capital, instrument_weights, fx_rate, idm)

    backtest_metrics = calculate_backtest_metrics(data, capital, strat)
    
    pd.options.display.float_format = '{:.2f}'.format  # Adjust precision as needed
    # Print the calculated metrics
    print(backtest_metrics.round(2))

    if plot==True:
        plot_position_size(data["Position Size"], data['Close'])
        plot_position_size_vs_pnl(data["Position Size"], data['Close'])


    return data


In [268]:
data = backtest(plot=False)
# data

[*********************100%%**********************]  1 of 1 completed

                              EWMAC_4_8_16_32_64
Initial Capital                        100000.00
Mean Annualised Return  %                   4.53
Avg Drawdown %                           -433.69
Annual Standard Deviation %              2118.23
Sharpe Ratio                                2.14
Turnover                                  110.44
Skew                                        0.78
Lower Tail (5th percentile)              -176.28
Upper Tail (95th percentile)              239.81
Cumulative PnL                          17925.01


In [246]:
ps, px = data['Position Size'], data['Close']

In [247]:
pnl = (ps * px.pct_change() * 1000).dropna().replace(0.0, np.nan).interpolate()
pnl

Date
2020-01-06     3.57
2020-01-07    -4.83
2020-01-08    18.38
2020-01-09     1.16
2020-01-10    10.96
              ...  
2023-12-22     1.14
2023-12-26    33.74
2023-12-27   -21.68
2023-12-28    -3.18
2023-12-29    26.53
Length: 1002, dtype: float64

In [251]:
np.log(pnl).pct_change().std()

c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log

C:\Users\karan\AppData\Local\Temp\ipykernel_14964\1982602473.py:1: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



2.280795863134806

In [257]:
std = np.log(pnl / pnl.shift(1)).replace(0, np.nan).interpolate().mean() * 252

std

c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log



6.019583135972088

In [265]:
import numpy as np
import pandas as pd

def calculate_position_size(capped_signals, risk_target, volatility, price, capital, instrument_weights, fx_rate, idm):
    """
    Calculate the position size based on capped signals, risk target, and other parameters.

    :param capped_signals: DataFrame containing capped EWMAC signals.
    :param risk_target: Annualized risk target.
    :param volatility: Current volatility of the instrument.
    :param price: Current price of the instrument.
    :param capital: Total capital available for trading.
    :param instrument_weights: Weights of instruments in the portfolio.
    :param fx_rate: Exchange rate if applicable.
    :param idm: Instrument Diversification Multiplier.
    :return: Position size.
    """
    multiplier = 1000
    # Calculate the average of capped signals
    average_signal = capped_signals.mean(axis=1)
    
    # Calculate risk-adjusted unrounded position
    position = average_signal * capital * idm * instrument_weights * risk_target / (10 * multiplier * price * fx_rate * volatility)
    
    return position

def calculate_pnl(position_size, prices, multiplier=1000):
    """
    Calculate profit and loss based on position size and prices.

    :param position_size: Series containing position size data.
    :param prices: Series containing price data.
    :param multiplier: Multiplier for position size.
    :return: Series containing P&L data.
    """
    return (position_size * prices * multiplier).replace(0.0, np.nan).dropna()

def calculate_backtest_metrics(data, capital, strat):
    """
    Calculate various backtest metrics based on position size and price data.

    :param data: DataFrame containing position size and daily returns.
    :param capital: Initial capital.
    :param strat: Strategy name.
    :return: DataFrame containing backtest metrics.
    """
    # Calculate P&L (Profit and Loss)
    pnl = calculate_pnl(data['Position Size'], data['Daily_Returns'])
    
    # Calculate cumulative P&L
    cumulative_pnl = pnl.cumsum()

    # Calculate Sharpe ratio
    sharpe_ratio = pnl.mean() / pnl.std() * (252 ** 0.5)

    # Calculate annualized return
    annualized_return = pnl.mean() * 252
    annualized_return_pct = annualized_return * 100 / capital

    # Additional metrics
    drawdowns = cumulative_pnl - cumulative_pnl.cummax()
    avg_drawdown = drawdowns.mean()

    # Calculate average position size
    avg_position = data['Position Size'].abs().mean()
    turnover = (data['Position Size'].diff().abs() / avg_position).sum()
    skew = pnl.skew()
    lower_tail = pnl.quantile(0.05)
    upper_tail = pnl.quantile(0.95)
    annual_std_dev = np.log(pnl / pnl.shift(1)).replace([np.inf, -np.inf], np.nan).dropna().std() * np.sqrt(252)
    alpha = (cumulative_pnl.iloc[-1] / cumulative_pnl.std()) * np.sqrt(252)
    
    return pd.DataFrame({
        "Initial Capital": [capital],
        "Mean Annualised Return %": [annualized_return_pct],
        "Avg Drawdown %": [avg_drawdown],
        "Annual Standard Deviation %": [annual_std_dev * 100],
        "Sharpe Ratio": [sharpe_ratio],
        "Turnover": [turnover],
        "Skew": [skew],
        "Lower Tail (5th percentile)": [lower_tail],
        "Upper Tail (95th percentile)": [upper_tail],
        "Cumulative PnL": [cumulative_pnl.iloc[-1]]
    }, index=[strat]).T

def backtest(ticker='BZ=F', start_date='2020-01-01', end_date='2024-01-01', parameters=[4, 8, 16, 32, 64],
             risk_target=0.1, capital=100000, plot=True):
    strat = "EWMAC_" + '_'.join([str(i) for i in parameters])
    instrument_weights = 1  # Example instrument weights
    fx_rate = 1.0  # Example exchange rate if applicable
    idm = 1  # Example Instrument Diversification Multiplier

    # Fetch data
    data = fetch_data(ticker, start_date, end_date)

    # Calculate daily returns
    data['Daily_Returns'] = data['Close'].pct_change()

    # Calculate volatility
    data['Volatility'] = data['Daily_Returns'].ewm(span=30, min_periods=0).std()

    # Calculate EWMAC signals (scaled and capped)
    data = calculate_ewmac_signals(data, parameters)
    signal_cols = [i for i in data.columns if "EWM" in i]

    # Calculate position size
    data["Position Size"] = calculate_position_size(data[signal_cols], risk_target, data['Volatility'], data['Close'], capital, instrument_weights, fx_rate, idm)

    backtest_metrics = calculate_backtest_metrics(data, capital, strat)
    
    pd.options.display.float_format = '{:.2f}'.format  # Adjust precision as needed
    # Print the calculated metrics
    print(backtest_metrics.round(2))

    if plot:
        plot_position_size(data["Position Size"], data['Close'])
        plot_position_size_vs_pnl(data["Position Size"], data['Close'])

    return data
backtest(plot=False)

[*********************100%%**********************]  1 of 1 completed

                              EWMAC_4_8_16_32_64
Initial Capital                        100000.00
Mean Annualised Return %                    4.53
Avg Drawdown %                           -433.69
Annual Standard Deviation %              2623.73
Sharpe Ratio                                2.14
Turnover                                  110.44
Skew                                        0.78
Lower Tail (5th percentile)              -176.28
Upper Tail (95th percentile)              239.81
Cumulative PnL                          17925.01



c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log



,Open,High,Low,Close,Adj Close,Volume,Daily_Returns,Volatility,EWMAC_4,EWMAC_8,EWMAC_16,EWMAC_32,EWMAC_64,Position Size
Date,,,,,,,,,,,,,,
2020-01-02,66.47,66.57,65.73,66.25,66.25,25810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,66.29,69.48,66.22,68.60,68.60,78848,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,69.07,70.72,68.19,68.91,68.91,51933,0.00,0.02,1.73,0.65,0.23,0.08,0.03,0.36
2020-01-07,68.58,68.74,67.65,68.27,68.27,41178,-0.01,0.02,1.12,0.50,0.19,0.07,0.03,0.25
2020-01-08,68.47,71.99,64.94,65.44,65.44,85232,-0.04,0.03,-1.47,-0.38,-0.10,-0.03,-0.01,-0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,79.44,80.37,78.83,79.07,79.07,12334,-0.00,0.02,4.82,-5.73,-11.27,-7.55,-3.98,-2.92
2023-12-26,79.19,81.71,78.81,81.07,81.07,10247,0.03,0.02,7.86,-3.36,-10.00,-7.18,-3.92,-1.97
2023-12-27,80.74,81.32,79.49,79.65,79.65,8282,-0.02,0.02,6.73,-2.65,-9.45,-7.11,-4.00,-2.00
